# Odd & Even Measurements

Assume you're given a table with measurement values obtaied from a Google sensor over multiple days with measurements taken multiple times within each day.

Write a query to calculate the sum of odd-numbered & even-numbered measurements separately for a particular day & display the results in two different columns, one for odd sum & another for even sums.

Definition:

* Within a day, measurements taken at 1st, 3rd, & 5th times are considered odd-numbered measurements, & measurements taken at 2nd, 4th & 6th times are considered even-numbered measurements.

# Answer

This is the `measurements` table.

```
CREATE TABLE measurements (
	measurement_id integer,
	measurement_value numeric(6, 2),
	measurement_time timestamp
);

COPY measurements
FROM '/YourDirectory/measurements.csv'
WITH (FORMAT CSV, HEADER);

SELECT * FROM measurements;
```

<img src = "measurements Table.png" width = "600" style = "margin:auto"/>

First, I'll rank by the measurements by the `measurement_time` to number the measurements in a day.

```
SELECT date_trunc('day', measurement_time),
	   rank() OVER (PARTITION BY date_part('day', 
	   	   measurement_time) ORDER BY measurement_time),
	   measurement_value
FROM measurements
ORDER BY date_part('day', measurement_time);
```

<img src = "Measurements Ranked by Timestamp.png" width = "600" style = "margin:auto"/>

With this view, I'll perform the groupings & conditional sums.

```
WITH measurement_ranked
AS (
	SELECT date_trunc('day', measurement_time),
		   rank() OVER (PARTITION BY date_part('day', 
		   	   measurement_time) ORDER BY 
			   measurement_time), 
		   measurement_value
	FROM measurements
	ORDER BY date_part('day', measurement_time)
)
SELECT date_trunc AS day,
	   sum(CASE WHEN rank % 2 != 0 THEN measurement_value
	   	   ELSE NULL END) AS odd_sums,
	   sum(CASE WHEN rank % 2 = 0 THEN measurement_value
	   	   ELSE NULL END) AS even_sums
FROM measurement_ranked
GROUP BY date_trunc
ORDER BY date_trunc;
```

<img src = "Odd & Even Sum of Measurements By Day.png" width = "600" style = "margin:auto"/>

On 7/10/2022, the sum of the odd-numbered measurements is 2355.75, while the sum of the even-numbered measuremenets is 1662.74. On 7/11/2022, the sum of the odd-numbered measurements is 2377.12, while the sum of the even-numbered measurements is 2480.70. On 7/12/2022, the sum of the odd-numbered measurements is 2903.40, while the sum of the even-numbered measurements is 1244.30.